In [2]:
import networkx as nx
import pandas as pd
from networkx.drawing.nx_agraph import graphviz_layout
import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

#note: for graphviz, need to install it separately, and also install the pygraphviz package.
#also, for plotly need to sign up and set the api key. refer to plotly set up guide on their website

Task 2 V